In [3]:
!pip install geopandas
import geopandas as gpd
from datetime import date

dir_geojson = "drive/My Drive/GRASP_EARTH/aois/los-angeles.geojson"
bucket = "ge-los-angeles"
orbit = "DESCENDING"

start_date = date(2021, 1, 1).strftime('%Y-%m-%d')
end_date = date(2021, 10, 15).strftime('%Y-%m-%d')

     |████████████████████████████████| 1.0 MB 5.4 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 6.3 MB 36.7 MB/s 


In [ ]:
import ee
import numpy as np
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
gdf = gpd.read_file(dir_geojson)
p = gdf["geometry"][0]
polygon = list(zip(*p.exterior.coords.xy))
aoi = ee.Geometry.Polygon(polygon)

In [10]:
if orbit in ["ASCENDING", "DESCENDING"]:
    Sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
                .filterBounds(aoi)\
                .filterDate(start_date, end_date)\
                .filter(ee.Filter.eq('orbitProperties_pass', orbit))\
                .select(['VV'])
else:
    Sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
                .filterBounds(aoi)\
                .filterDate(start_date, end_date)\
                .select(['VV'])

img_list = Sentinel1.toList(200)

## 取得したデータ数の確認

In [8]:
img_num = img_list.size().getInfo()
img_num

46

## Download image

In [9]:
for i in range(img_list.size().getInfo()):
    image = ee.Image(img_list.get(i))
    task = ee.batch.Export.image.toCloudStorage(
        image=image.reproject(crs='EPSG:4326',scale=10),
        description=image.get('system:index').getInfo(),
        bucket=bucket,
        region=aoi,
        scale=10
        )
    task.start()